## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSSSBB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1271
+-------------------+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|            coeff_1|           coeff_2|           coeff_3|           coeff_4|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|              res_4|           res_mean|    station|  total_var|undefs|              vector|  year|
+-------------------+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|-1073.8061998945723| 425.3767238176554| 42.68208887773061|3.

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [25]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1 FROM weather


,station,year,coeff_1
0,USC00040924,1965.0,-1073.806200
1,USC00025467,1946.0,867.690569
2,USC00029464,1943.0,1144.772608
3,USC00025467,1973.0,1012.426690
4,USC00021026,1991.0,-1123.666290


In [27]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station,USC00020060,USC00020104,USC00020406,USC00020949,USC00021026,USC00021161,USC00021282,USC00021353,USC00021356,USC00021361,...,USC00029634,USC00040924,USC00042598,USC00043855,USC00044259,USC00045502,USC00045860,USC00046635,USW00023183,USW00093140
year,,,,,,,,,,,,,,,,,,,,,
1901.0,NaN,NaN,NaN,NaN,1445.293268,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1902.0,NaN,NaN,NaN,NaN,1690.212935,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-68.682108,NaN,NaN,NaN,NaN,NaN
1908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [28]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
print mean_by_year
mean_by_station=np.nanmean(year_station_table,axis=0)
print mean_by_station
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

[ 1192.21534971  1278.25465438  1038.82196331   997.26105039   865.16136243
   371.15442029   207.97051026   574.90486981   493.96488132   224.19942378
   849.45302563   718.47652077   367.5028109     78.57794847   444.43932114
   198.07274462   464.24177385   842.4433561    514.09407544   776.39040259
   539.98614263   391.52975629   338.83026132   481.24394183   290.26929126
    62.56503724   202.3010922    336.25423416   371.5953304    439.24401488
   330.32914802   684.60553906   479.25355357   232.43639592   436.4186631
   246.15740064   350.86324746   194.32316624   285.22030354   327.37380147
   543.15373305   322.55392178   212.88449646   420.19761363   476.95276304
   356.20144783   264.93464445   503.64135997    65.12132314  -197.71308538
    52.63318975   146.54875548   135.73295031  -134.00882215   183.90278178
   -32.55910832    95.77686998    42.91242733   170.58916048   249.1724115
   102.32486566    72.64058572   -33.14085447   170.22478352   -10.12056117
  -125.4456890

In [17]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 187.183429585
0 after removing mean by year    = 131.36786219
0 after removing mean by stations= 127.456228588
1 after removing mean by year    = 127.321951461
1 after removing mean by stations= 127.307980934
2 after removing mean by year    = 127.304833195
2 after removing mean by stations= 127.30374902
3 after removing mean by year    = 127.303287503
3 after removing mean by stations= 127.303070806
4 after removing mean by year    = 127.302964717
4 after removing mean by stations= 127.302911883
